In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

Using TensorFlow backend.
/home/kasarov/.virtualenvs/tf/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_excel('../dataset/default_of_credit_card_clients.xls', skiprows=1, index_col=0)

In [3]:
df.head(2)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1


#### Conversão de variáveis de tipo categóricas para one-hot

In [4]:
# Sex
df = df.assign(MALE=(df.SEX == 1).astype(np.int32))
# df = df.assign(FEMALE=(df.SEX == 2).astype(np.int32))

In [5]:
# Marital status
df = df.assign(MS_SINGLE=(df.MARRIAGE == 2).astype(np.int32))
df = df.assign(MS_MARRIED=(df.MARRIAGE == 1).astype(np.int32))
df = df.assign(MS_OTHERS=(df.MARRIAGE == 3).astype(np.int32))

In [6]:
df.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,MALE,MS_SINGLE,MS_MARRIED,MS_OTHERS
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,...,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200,0.396267,0.532133,0.455300,0.010767
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,...,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062,0.489129,0.498975,0.498006,0.103204
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df = df.assign(cat1=df.LIMIT_BAL < 50000)
df = df.assign(cat2=(df.LIMIT_BAL >= 50000) & (df.LIMIT_BAL < 140000))
df = df.assign(cat3=(df.LIMIT_BAL >= 140000) & (df.LIMIT_BAL < 240000))
df = df.assign(cat4=df.LIMIT_BAL >= 240000)

In [8]:
duly_tendency = []
def get_duly_tendency(row):
    aux = []
    for i in range(6):
        aux.append(row[i+5])
    duly_tendency.append(np.mean(aux))

In [9]:
df_matrix = df.as_matrix()
np.apply_along_axis(get_duly_tendency, 1, df)
df['duly_tend'] = duly_tendency

#### Extração dos dados

In [10]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month', 'MALE', 'MS_SINGLE', 'MS_MARRIED',
       'MS_OTHERS', 'cat1', 'cat2', 'cat3', 'cat4', 'duly_tend'],
      dtype='object')

In [11]:
x = df[['LIMIT_BAL', 'EDUCATION', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4',
       'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'MALE', 'MS_SINGLE', 'MS_MARRIED', 'MS_OTHERS', 'duly_tend']]
# y = df[['cat1', 'cat2', 'cat3', 'cat4']]
y = df[['default payment next month']]

In [12]:
x.head()

,LIMIT_BAL,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,MALE,MS_SINGLE,MS_MARRIED,MS_OTHERS,duly_tend
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,24,2,2,-1,-1,-2,-2,3913,...,689,0,0,0,0,0,0,1,0,-0.333333
2,120000,2,26,-1,2,0,0,0,2,2682,...,1000,1000,1000,0,2000,0,1,0,0,0.500000
3,90000,2,34,0,0,0,0,0,0,29239,...,1500,1000,1000,1000,5000,0,1,0,0,0.000000
4,50000,2,37,0,0,0,0,0,0,46990,...,2019,1200,1100,1069,1000,0,0,1,0,0.000000
5,50000,2,57,-1,0,-1,0,0,0,8617,...,36681,10000,9000,689,679,1,0,1,0,-0.333333


In [13]:
y.head()

,default payment next month
ID,
1,1
2,1
3,0
4,0
5,0


In [14]:
x = x.as_matrix().astype(np.float32)
y = y.as_matrix().astype(np.float32)

In [15]:
x[:,0] = x[:,0] / 1000.

In [16]:
x[:,2] = x[:,2] / 10.

In [17]:
x[:, 9:21]

array([[  3913.,   3102.,    689., ...,      0.,      0.,      0.],
       [  2682.,   1725.,   2682., ...,   1000.,      0.,   2000.],
       [ 29239.,  14027.,  13559., ...,   1000.,   1000.,   5000.],
       ..., 
       [  3565.,   3356.,   2758., ...,   4200.,   2000.,   3100.],
       [ -1645.,  78379.,  76304., ...,   1926.,  52964.,   1804.],
       [ 47929.,  48905.,  49764., ...,   1000.,   1000.,   1000.]], dtype=float32)

In [18]:
x[:,9:21] = x[:,9:21] / 1000.

In [19]:
x[:, 9:21]

array([[  3.91300011,   3.102     ,   0.68900001, ...,   0.        ,
          0.        ,   0.        ],
       [  2.68199992,   1.72500002,   2.68199992, ...,   1.        ,
          0.        ,   2.        ],
       [ 29.23900032,  14.02700043,  13.55900002, ...,   1.        ,
          1.        ,   5.        ],
       ..., 
       [  3.56500006,   3.35599995,   2.7579999 , ...,   4.19999981,
          2.        ,   3.0999999 ],
       [ -1.64499998,  78.3789978 ,  76.30400085, ...,   1.926     ,
         52.9640007 ,   1.80400002],
       [ 47.92900085,  48.90499878,  49.76399994, ...,   1.        ,
          1.        ,   1.        ]], dtype=float32)

In [20]:
# p = np.random.permutation(len(x))
lim = np.floor(.8*len(x)).astype(int)

In [21]:
# X_train, X_test = x[p[:lim]], x[p[lim:]]
# y_train, y_test = y[p[:lim]], y[p[lim:]]
X_train, X_test = x[:lim], x[lim:]
y_train, y_test = y[:lim], y[lim:]

In [22]:
X_train.shape

(24000, 26)

In [23]:
y_test.shape

(6000, 1)

#### Criação do modelo

In [24]:
model = Sequential()

In [25]:
model.add(Dense(128, activation='relu', input_dim=x.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(2**y_test.shape[1], activation='softmax'))

In [26]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, to_categorical(y_train), epochs=10, batch_size=32, validation_data=(X_test, to_categorical(y_test)))

Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 2s 75us/step - loss: 0.5424 - categorical_accuracy: 0.7608 - val_loss: 0.4851 - val_categorical_accuracy: 0.7913
Epoch 2/10
24000/24000 [==============================] - 2s 69us/step - loss: 0.5084 - categorical_accuracy: 0.7808 - val_loss: 0.4621 - val_categorical_accuracy: 0.8048
Epoch 3/10
24000/24000 [==============================] - 2s 68us/step - loss: 0.4878 - categorical_accuracy: 0.7910 - val_loss: 0.4464 - val_categorical_accuracy: 0.8147
Epoch 4/10
24000/24000 [==============================] - 2s 69us/step - loss: 0.4775 - categorical_accuracy: 0.7962 - val_loss: 0.4361 - val_categorical_accuracy: 0.8182
Epoch 5/10
24000/24000 [==============================] - 2s 90us/step - loss: 0.4696 - categorical_accuracy: 0.8007 - val_loss: 0.4373 - val_categorical_accuracy: 0.8185
Epoch 6/10
24000/24000 [==============================] - 2s 81us/step - loss: 0.4694 - categori

In [28]:
score = model.evaluate(X_test, to_categorical(y_test), batch_size=32)
score

6000/6000 [==============================] - 0s 33us/step


[0.42677622286478678, 0.82250000000000001]

In [29]:
to_categorical(model.predict_classes(X_test))

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [30]:
np.sum(np.power(y_test.reshape(6000) - model.predict_classes(X_test), 2))/y_test.shape[0]

0.17749999999999999

In [31]:
y_test.reshape(6000)

array([ 0.,  1.,  0., ...,  1.,  1.,  1.], dtype=float32)

In [32]:
model.predict_classes(X_test)

array([1, 0, 0, ..., 1, 0, 0])

In [33]:
np.sum(np.power(y_train.reshape(24000) - model.predict_classes(X_train), 2))/y_train.shape[0]

0.18987499999999999

### Variação paramétrica

In [34]:
def bissecao(model, X):
    n = len(X)
    a = X[0]
    ca = model.predict_classes(np.reshape(X, [1, n]))
    b = 2*a
    X[0] = b

    # encontra o primeiro valor a cruzar o zero
    cb = model.predict_classes(np.reshape(X, [1, n]))
    if cb == 1:
        i = 0
        while cb == 1 and i <= 2: # enquanto for pagador
            b = 2*b
            X[0] = b
            cb = model.predict_classes(np.reshape(X, [1, n]))
            i += 1
        if cb == 1 and i == 3:
            return a, b # não queremos aumentar tanto assim o limite
    else:
        i = 0
        while ca == 0 and a > 1.:
            a = a/2
            X[0] = a
            ca = model.predict_classes(np.reshape(X, [1,n]))
            i += 1
        if ca == 0:
            return a, b


    i = 0
    while ((b - a) >= 10) and i < 50:
        p = a + (b-a)/2
        cp = model.predict_classes(np.reshape(X, [1, n]))
        if ca == 1 and cp == 1 or ca == 0 and cp == 0:
            a = p
            ca = cp
        else:
            b = p
            X[0] = b
        i += 1

    return a, b

In [35]:
X0_mod = np.copy(X_test[0])

In [36]:
bissecao(model=model, X=X0_mod)

(115.625, 125.0)

In [37]:
X0_mod[0] = 137.5

In [38]:
model.predict_classes(np.reshape(X0_mod, [1, 26]))

array([1])

# Como melhorar meu crédito?

In [ ]:
X2_mod = np.copy(X_train[24])
X2_mod

X2_mod[3:21] = X_train[0,3:21]

X2_mod[3:9].fill(-1.)

In [ ]:
model.predict_classes(np.reshape(X2_mod, [1, 26]))

In [ ]:
X2_mod[9:15] = np.zeros(6)
X2_mod[9:15].fill(1.)

X2_mod[9:15] += 29239/6

In [ ]:
X2_mod[15:21] = X2_mod[9:15]/2.

In [ ]:
X2_mod[1] = 4.
X2_mod[2] = 3.4

In [ ]:
X2_mod = X2_mod.copy()
X2_mod

In [ ]:
model.predict_classes(np.reshape(X2_mod, [1, 26]))

In [ ]:
bissecao(model, X2_mod)

In [ ]:
X2_mod[0] = 174.375
X2_mod = X2_mod.copy()
model.predict_classes(np.reshape(X2_mod, [1,26]))

In [ ]:
X2_mod[0] = 180.
X2_mod = X2_mod.copy()
model.predict_classes(np.reshape(X2_mod, [1,26]))

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 33 columns):
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                

In [ ]:
def optmize_credit(row):
    X = row[1:]
    old_a, old_b = bissecao(row, X)
    old_avg = (old_a+old_b)/2
    unchangeable = [0, 1, 2, 3, 4, ]
    for i in range(6):
        if X[i+4]>0:
            if (row[]/row[])<0.5:
                new_a, 

### Exportação de modelo

In [39]:
model.save('model.h5')

In [40]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)